In [1]:
import lightgbm as lgb
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
import gc

In [2]:
import time

# Function to make presiction and save it to csv file in kaggle submission format
def save_submission(model, extension="", transform=None):
    
    if transform is None:
        # Converting predictions into exp(x) - 1
        pred = np.expm1(model.predict(test.drop(columns=["timestamp","row_id"])))
    else:
        pred = np.expm1(model.predict(transform(test.drop(columns=["timestamp","row_id"]))))
        
    filename = time.strftime("%y-%m-%d-%H%M%S") + extension +".csv"

    # Saving predictions to csv file
    pd.DataFrame(pred, columns=["meter_reading"]).rename_axis("row_id").to_csv(filename)
    
    #converts the csv file to 7z to reduce size
    !p7zip $filename >> /dev/null
    print("{} saved!".format(filename))

# save model to disk in pickle format
def save_model(model, name="model.pkl"):
    with open("../models/" + name, "wb") as f:
        pickle.dump(model, f)
    print("{} saved to disk!".format(name))

In [3]:
with open("../data/train_featurized.pkl", "rb") as f:
    train = pickle.load(f)

In [4]:
train

,building_id,meter,timestamp,meter_reading,DT_M,DT_W,DT_D,DT_hour,DT_day_week,DT_day_month,...,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,105,0,2016-01-01 00:00:00,23.3036,1,53,1,0,4,1,...,0,50623,-999,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
1,106,0,2016-01-01 00:00:00,0.3746,1,53,1,0,4,1,...,0,5374,-999,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
2,106,3,2016-01-01 00:00:00,0.0000,1,53,1,0,4,1,...,0,5374,-999,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
3,107,0,2016-01-01 00:00:00,175.1840,1,53,1,0,4,1,...,0,97532,2005,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
4,108,0,2016-01-01 00:00:00,91.2653,1,53,1,0,4,1,...,0,81580,1913,3.8,NaN,2.4,NaN,1020.9,240.0,3.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19811273,1444,0,2016-12-31 23:00:00,8.7500,12,52,366,23,5,31,...,1,19619,1914,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19811274,1445,0,2016-12-31 23:00:00,4.8250,12,52,366,23,5,31,...,0,4298,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19811275,1446,0,2016-12-31 23:00:00,0.0000,12,52,366,23,5,31,...,1,11265,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19811276,1447,0,2016-12-31 23:00:00,159.5750,12,52,366,23,5,31,...,10,29775,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
X = train.drop(columns=["timestamp","meter_reading"])

# converting meter_reading to y where y=log(meter_reading + 1)
y = np.log1p(train["meter_reading"])

In [6]:
from sklearn.metrics import make_scorer

def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log(y_pred+1) - np.log(y_true+1))))

rmsle_error = make_scorer(rmsle, greater_is_better=False)

### Lgbm Model

In [12]:
param_grid = {
    "learning_rate" : [0.01],
    "n_estimators" : [512, 1024],
    "num_leaves" : [32,64],
    "objective" : ["regression"],
    "metric" : ["rmse"],
    "colsample_bytree": [0.9],
}

lgbm = LGBMRegressor()
grid = GridSearchCV(lgbm, param_grid, scoring="neg_mean_squared_error", cv=3, return_train_score=True)
grid.fit(X, y)

GridSearchCV(cv=3, estimator=LGBMRegressor(),
             param_grid={'colsample_bytree': [0.9], 'learning_rate': [0.01],
                         'metric': ['rmse'], 'n_estimators': [512, 1024],
                         'num_leaves': [32, 64], 'objective': ['regression']},
             return_train_score=True, scoring='neg_mean_squared_error')

In [13]:
grid.cv_results_

{'mean_fit_time': array([199.72734555, 229.77577527, 348.01061145, 387.6663216 ]),
 'std_fit_time': array([4.19531339, 4.42991553, 6.98625899, 5.69784995]),
 'mean_score_time': array([ 40.16014163,  53.91560276,  99.09592072, 120.87006418]),
 'std_score_time': array([0.67412087, 0.61536504, 1.0521504 , 1.14858308]),
 'param_colsample_bytree': masked_array(data=[0.9, 0.9, 0.9, 0.9],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array(data=[0.01, 0.01, 0.01, 0.01],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_metric': masked_array(data=['rmse', 'rmse', 'rmse', 'rmse'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[512, 512, 1024, 1024],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n

In [14]:
grid.best_params_

{'colsample_bytree': 0.9,
 'learning_rate': 0.01,
 'metric': 'rmse',
 'n_estimators': 1024,
 'num_leaves': 64,
 'objective': 'regression'}

In [16]:
features = ['building_id', 'meter', 'DT_M', 'DT_W', 'DT_D', 'DT_hour',
       'DT_day_week', 'DT_day_month', 'DT_week_month', 'DT_h_sin', 'DT_h_cos',
       'weekend', 'site_id', 'primary_use', 'square_feet', 'year_built',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed']

categorical_features = ['building_id', 'meter', 'DT_M', 'DT_W',
       'DT_D', 'DT_hour', 'DT_day_week', 'DT_day_month', 'DT_week_month', 'weekend',
       'site_id', 'primary_use', 'year_built']

train_data = lgb.Dataset(X,label=y, categorical_feature=categorical_features, feature_name=features)

In [17]:
param= {
    "learning_rate" : 0.01,
    "num_iterations" : 1024,
    "num_leaves" : 64,
    "objective" : "regression",
    "metric" : "rmse",
    "colsample_bytree": 0.9,

}

lgbm = lgb.train(param, train_data, categorical_feature=categorical_features, feature_name=features)

save_submission(lgbm, "lgbm")
save_model(lgbm,"lgbm.pkl")

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.730629 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3237
[LightGBM] [Info] Number of data points in the train set: 19811278, number of used features: 23
[LightGBM] [Info] Start training from score 4.212522
21-02-23-193333lgbm.csv saved!
lgbm.pkl saved to disk!


<img src="../images/model/kaggle/lgbm.png">

### Imputation with median

In [7]:
X.isnull().sum()

building_id                 0
meter                       0
DT_M                        0
DT_W                        0
DT_D                        0
DT_hour                     0
DT_day_week                 0
DT_day_month                0
DT_week_month               0
DT_h_sin                    0
DT_h_cos                    0
weekend                     0
site_id                     0
primary_use                 0
square_feet                 0
year_built                  0
air_temperature        107896
cloud_coverage        8644205
dew_temperature        111378
precip_depth_1_hr     3757868
sea_level_pressure    1234975
wind_direction        1450706
wind_speed             155227
dtype: int64

In [8]:
# all these are numerical features
features_to_impute = [
    "air_temperature",
    "cloud_coverage",
    "dew_temperature",
    "precip_depth_1_hr",
    "sea_level_pressure",
    "wind_direction",
    "wind_speed"
]

impute_vals = {}

# Median imputation

for site_id in range(0,15+1):
    site = {}
    for feature in features_to_impute:
        median = X[X["site_id"]==site_id][feature].median()
        median = 0 if np.isnan(median) else median
        site.update({feature : median})
      
    impute_vals.update({site_id : site})

In [9]:
impute_vals

{0: {'air_temperature': 25.0,
  'cloud_coverage': 4.0,
  'dew_temperature': 21.7,
  'precip_depth_1_hr': 0.0,
  'sea_level_pressure': 1018.0,
  'wind_direction': 120.0,
  'wind_speed': 3.1},
 1: {'air_temperature': 11.3,
  'cloud_coverage': 0.0,
  'dew_temperature': 7.8,
  'precip_depth_1_hr': 0,
  'sea_level_pressure': 1017.6,
  'wind_direction': 220.0,
  'wind_speed': 3.6},
 2: {'air_temperature': 25.6,
  'cloud_coverage': 2.0,
  'dew_temperature': 3.3,
  'precip_depth_1_hr': 0.0,
  'sea_level_pressure': 1011.0,
  'wind_direction': 120.0,
  'wind_speed': 2.6},
 3: {'air_temperature': 15.6,
  'cloud_coverage': 4.0,
  'dew_temperature': 9.4,
  'precip_depth_1_hr': 0.0,
  'sea_level_pressure': 1018.0,
  'wind_direction': 190.0,
  'wind_speed': 3.6},
 4: {'air_temperature': 15.0,
  'cloud_coverage': 2.0,
  'dew_temperature': 10.6,
  'precip_depth_1_hr': 0.0,
  'sea_level_pressure': 1016.7,
  'wind_direction': 260.0,
  'wind_speed': 3.6},
 5: {'air_temperature': 11.0,
  'cloud_coverage': 

In [10]:
# filling null values
for site_id in range(0, 15+1):
    df = X[X["site_id"]==site_id].fillna(impute_vals[site_id])
    X[X["site_id"]==site_id] = df

In [11]:
# checking for null vals after imputation
X.isnull().sum()

building_id           0
meter                 0
DT_M                  0
DT_W                  0
DT_D                  0
DT_hour               0
DT_day_week           0
DT_day_month          0
DT_week_month         0
DT_h_sin              0
DT_h_cos              0
weekend               0
site_id               0
primary_use           0
square_feet           0
year_built            0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [15]:
# filling null vals in test data from train data impute vals
with open("../data/test_featurized.pkl", "rb") as f:
    test = pickle.load(f)
    
for site_id in range(0, 15+1):
    df = test[test["site_id"]==site_id].fillna(impute_vals[site_id])
    test[test["site_id"]==site_id] = df

del df
gc.collect()

0

### Linear model

In [24]:
param_grid = {
    'alpha' :[0.0001, 0.01, 0.1]
}

lin = SGDRegressor()
grid = GridSearchCV(lin, param_grid, scoring="neg_mean_squared_error", cv=3)
grid.fit(X ,y)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:1211: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


GridSearchCV(cv=3, estimator=SGDRegressor(),
             param_grid={'alpha': [0.0001, 0.01, 0.1]},
             scoring='neg_mean_squared_error')

In [25]:
grid.best_params_

{'alpha': 0.01}

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

linear = SGDRegressor(alpha = 0.01, eta0=1e-5, learning_rate="constant")
linear.fit(scaler.fit_transform(X) ,y)

del X
del train
del y
gc.collect()

save_submission(linear, "_lin", scaler.transform)
save_model(linear,"linear.pkl")

21-02-26-112301_lin.csv saved!
linear.pkl saved to disk!


<img src="../images/model/kaggle/lin.png">

### Decision tree

In [29]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X, y)

del X
del train
del y
gc.collect()

save_submission(dtr, "_dtr")
save_model(dtr,"dtr.pkl")

21-02-25-181850_dtr.csv saved!
dtr.pkl saved to disk!


<img src="../images/model/kaggle/dtr.png">

### Ada boost

In [30]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor(n_estimators = 1000)
ada.fit(X, y)

AdaBoostRegressor(n_estimators=1000)

In [31]:
del X
del train
del y
gc.collect()
save_model(ada,"ada.pkl")

ada.pkl saved to disk!


In [36]:
save_submission(ada, "_ada")

21-02-26-193019_ada.csv saved!


<img src="../images/model/kaggle/ada.png">

### MLP

In [12]:
import tensorflow as tf
import datetime

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.02, random_state=42)

model = tf.keras.Sequential([
    Dense(23, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dropout(0.2),
    Dense(4, activation='relu'),
    Dense(2, activation='relu'),
    Dense(1)
    ])

model.compile(optimizer='adam',
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=tf.keras.losses.MeanSquaredError())

model.fit(X_train, y_train)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(X, y, epochs=4, callbacks=[tensorboard_callback],batch_size = 512, validation_data=(X_val, y_val), validation_batch_size=64)

606721/606721 [==============================] - 1877s 3ms/step - loss: 5.0854 - mean_squared_error: 5.0854
Epoch 1/4
38694/38694 [==============================] - 348s 9ms/step - loss: 4.3686 - mean_squared_error: 4.3686 - val_loss: 4.3637 - val_mean_squared_error: 4.3644
Epoch 2/4
38694/38694 [==============================] - 340s 9ms/step - loss: 4.3686 - mean_squared_error: 4.3686 - val_loss: 4.3637 - val_mean_squared_error: 4.3644
Epoch 3/4
38694/38694 [==============================] - 345s 9ms/step - loss: 4.3686 - mean_squared_error: 4.3686 - val_loss: 4.3637 - val_mean_squared_error: 4.3644
Epoch 4/4
38694/38694 [==============================] - 340s 9ms/step - loss: 4.3686 - mean_squared_error: 4.3686 - val_loss: 4.3638 - val_mean_squared_error: 4.3645


In [13]:
model.save('mlp')

INFO:tensorflow:Assets written to: mlp/assets


In [15]:
save_submission(model, "_mlp")

21-03-01-181751_mlp.csv saved!


<img src="../images/model/kaggle/mlp.png">

### Stacking

In [12]:
with open("ada.pkl", "rb") as f:
    ada = pickle.load(f)

with open("lgbm.pkl", "rb") as f:
    lgbm = pickle.load(f)

with open("linear.pkl", "rb") as f:
    linear = pickle.load(f)
    
def return_train_prediction(model, transform=None):
    
    if transform is None:
        return model.predict(X)
    else:
        return model.predict(transform(X))

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
        
base = [ada, lgbm, linear]
transforms=[None, None, scaler.transform]
meta = SGDRegressor(alpha = 0.01, eta0=1e-5, learning_rate="constant")

base_pred = np.zeros((X.shape[0],len(base)), dtype='float')

for i,(m,t) in enumerate(zip(base, transforms)):
    base_pred[:,i] = return_train_prediction(m, t)

meta.fit(base_pred, y)

SGDRegressor(alpha=0.01, eta0=1e-05, learning_rate='constant')

In [13]:
save_model(meta,"meta.pkl")

meta.pkl saved to disk!


In [16]:
base_pred = np.zeros((test.shape[0],len(base)), dtype='float')
    
def return_test_prediction(model, transform=None):
    
    if transform is None:
        return model.predict(test.drop(columns=["timestamp","row_id"]))
    else:
        return model.predict(transform(test.drop(columns=["timestamp","row_id"])))
    
for i,(m,t) in enumerate(zip(base, transforms)):
    base_pred[:,i] = return_test_prediction(m, t)

filename = time.strftime("%y-%m-%d-%H%M%S") + "_meta" +".csv"
pd.DataFrame(np.expm1(meta.predict(base_pred)), columns=["meter_reading"]).rename_axis("row_id").to_csv(filename)
print(f"{filename} saved!")
!p7zip $filename >> /dev/null

21-03-03-140840_meta.csv saved!


<img src="../images/model/kaggle/meta.png">